###Fetching Dataset and Basic Preprocessing

In [21]:
from google.colab import drive
drive.mount('/content/drive/')
%cd "/content/drive/MyDrive/Colab Notebooks/OnCampus/LongCovid/regression/Data"

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
/content/drive/MyDrive/Colab Notebooks/OnCampus/LongCovid/regression/Data


In [22]:
# Import necessary libraries
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import accuracy_score, mean_squared_error, f1_score
from sklearn.preprocessing import label_binarize
import numpy as np
import pandas as pd

In [23]:
f_path = "HRV_features_60seconds_HRVDelta.xlsx"
og_data = pd.read_excel(f_path, header=0)
og_data.head()

,Block,fatigue_rating,window_num_60,SubjID,Age,HRV_MeanNN,HRV_SDNN,HRV_SDANN1,HRV_SDNNI1,HRV_SDANN2,...,Delta_HRV_SampEn,Delta_HRV_ShanEn,Delta_HRV_FuzzyEn,Delta_HRV_MSEn,Delta_HRV_CMSEn,Delta_HRV_RCMSEn,Delta_HRV_CD,Delta_HRV_HFD,Delta_HRV_KFD,Delta_HRV_LZC
0,0,73,0,Cov4,38,709.821429,121.764550,NaN,NaN,NaN,...,0.905825,3.068806,-3.022311,-0.442216,0.117807,0.543697,0.336966,0.015104,0.990893,-1.868155
1,0,73,1,Cov4,38,783.125000,92.079715,NaN,NaN,NaN,...,0.905825,3.068806,-3.022311,-0.442216,0.117807,0.543697,0.336966,0.015104,0.990893,-1.868155
2,0,73,2,Cov4,38,767.248377,132.329919,NaN,NaN,NaN,...,0.905825,3.068806,-3.022311,-0.442216,0.117807,0.543697,0.336966,0.015104,0.990893,-1.868155
3,0,73,3,Cov4,38,760.416667,111.418943,NaN,NaN,NaN,...,0.905825,3.068806,-3.022311,-0.442216,0.117807,0.543697,0.336966,0.015104,0.990893,-1.868155
4,0,73,4,Cov4,38,762.419872,109.225866,NaN,NaN,NaN,...,0.905825,3.068806,-3.022311,-0.442216,0.117807,0.543697,0.336966,0.015104,0.990893,-1.868155


In [24]:
og_data['SubjID'] = og_data['SubjID'].str.replace('Cov', '')
og_data['SubjID'] = og_data['SubjID'].astype(int)

In [25]:
og_data = og_data.dropna(axis=1, how='all')
og_data = og_data[~og_data.isin([np.nan, np.inf, -np.inf]).any(axis=1)]

In [26]:
og_data['Window_ID'] = og_data['SubjID'].map(str) + '_' + og_data['Block'].map(str) + '_' + og_data['window_num_60'].map(str)
og_data['Block_ID'] = og_data['SubjID'].map(str) + '_' + og_data['Block'].map(str)

In [27]:
block_zero = og_data['Block'] == 0
data = og_data[~block_zero]

###Decision Tree after Dropping SubjID

In [28]:
y = data['Change_from_Baseline']
X = data.drop(['fatigue_rating', 'Change_from_Baseline','Delta_fatigue'], axis=1)
X = X.drop('SubjID', axis=1)

# Define the indexes of blocks in your data based on blockid
blocks_indexes = [list(og_data[og_data['Block_ID'] == block_id].index) for block_id in og_data['Block_ID'].unique()]

In [29]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize the decision tree classifier
clf = DecisionTreeRegressor(random_state=42)

# Train the classifier on the training data
clf.fit(X_train, y_train)

# Make predictions on the testing data
y_pred = clf.predict(X_test)

# Calculate the accuracy of the classifier
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

# Calculate the RMSE (Root Mean Squared Error)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print("RMSE:", rmse)

# Calculate the F1 score
f1 = f1_score(y_test, y_pred, average='weighted')
print("F1 Score:", f1)

Accuracy: 0.8918918918918919
RMSE: 0.6575959492214292
F1 Score: 0.8918918918918919


###Parameter tuning and validation

In [30]:
from sklearn.model_selection import GridSearchCV

# Define the parameter grid to search
param_grid = {
    'max_depth': [3, 5, 7, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Initialize the decision tree classifier
clf = DecisionTreeRegressor(random_state=42)

# Initialize GridSearchCV
grid_search = GridSearchCV(estimator=clf, param_grid=param_grid, cv=5)

# Perform grid search
grid_search.fit(X_train, y_train)

# Get the best parameters and the best score
best_params = grid_search.best_params_
best_score = grid_search.best_score_

print("Best Parameters:", best_params)
print("Best Score:", best_score)

# Retrain the model with the best parameters
best_clf = grid_search.best_estimator_
best_clf.fit(X_train, y_train)

# Make predictions on the testing data using the best model
y_pred = best_clf.predict(X_test)

# Calculate the accuracy of the classifier
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

# Calculate the RMSE (Root Mean Squared Error)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print("RMSE:", rmse)

# Calculate the F1 score
f1 = f1_score(y_test, y_pred, average='weighted')
print("F1 Score:", f1)

Best Parameters: {'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 5}
Best Score: 0.8928073035443177
Accuracy: 0.8918918918918919
RMSE: 0.6575959492214292
F1 Score: 0.8918918918918919


Leave one subject out validation

In [31]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import LeaveOneGroupOut
from sklearn.metrics import accuracy_score

# Define features and target
sub_ids = og_data['SubjID'].unique()
print(sub_ids)
# Initialize Decision Tree classifier
clf = DecisionTreeClassifier()

for sub in sub_ids:
  print(sub)
  df = og_data[og_data['SubjID'] != sub]
  print(len(df))
  y = df['fatigue_rating']
  X = df.drop('fatigue_rating', axis=1)
  X = X.drop('SubjID', axis=1)
  # Split the dataset into training and testing sets
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

  # Initialize the decision tree classifier
  clf = DecisionTreeClassifier(random_state=42)

  # Train the classifier on the training data
  clf.fit(X_train, y_train)

  # Make predictions on the testing data
  y_pred = clf.predict(X_test)
  # Calculate the accuracy of the classifier
  accuracy = accuracy_score(y_test, y_pred)
  print("Accuracy:", accuracy)

  # Calculate the RMSE (Root Mean Squared Error)
  rmse = np.sqrt(mean_squared_error(y_test, y_pred))
  print("RMSE:", rmse)

  # Calculate the F1 score
  f1 = f1_score(y_test, y_pred, average='weighted')
  print("F1 Score:", f1)


[ 4  5  7  8 13 14 19 20 22 23]
4
205
Accuracy: 0.926829268292683
RMSE: 4.884020734659648
F1 Score: 0.9297909407665504
5
204
Accuracy: 0.926829268292683
RMSE: 2.1068992359018743
F1 Score: 0.9365853658536585
7
208
Accuracy: 0.9285714285714286
RMSE: 1.9639610121239315
F1 Score: 0.9206349206349207
8
205
Accuracy: 0.9512195121951219
RMSE: 7.190780004503735
F1 Score: 0.9414634146341464
13
224
Accuracy: 0.8888888888888888
RMSE: 1.0
F1 Score: 0.8857142857142858
14
217
Accuracy: 0.9090909090909091
RMSE: 15.454478609481107
F1 Score: 0.8909090909090908
19
206
Accuracy: 0.7619047619047619
RMSE: 25.908722561367338
F1 Score: 0.7401360544217688
20
200
Accuracy: 0.85
RMSE: 5.935065290289568
F1 Score: 0.8138888888888889
22
231
Accuracy: 1.0
RMSE: 0.0
F1 Score: 1.0
23
206
Accuracy: 0.8333333333333334
RMSE: 2.220038609702865
F1 Score: 0.7910052910052909


In [32]:
blocks_indexes = [list(X_test[X_test['Block_ID'] == block_id].index) for block_id in X_test['Block_ID'].unique()]
# Predict on test set
window_predictions = clf.predict(X_test)
accuracy = accuracy_score(y_test, window_predictions)
print("Accuracy:", accuracy)

# Create block_mapping_test
# For X_test
X_block_data_test = {}
for block_id, indexes in enumerate(blocks_indexes):
    X_block_data_test[block_id] = X_test.loc[indexes]

# For y_test
y_block_data_test = {}
for block_id, indexes in enumerate(blocks_indexes):
    y_block_data_test[block_id] = y_test.loc[indexes]

# For y_pred
y_block_data_pred = {}
for block_id, indexes in enumerate(blocks_indexes):
    y_block_data_pred[block_id] = y_test.loc[indexes]

# Aggregate window predictions to block level
block_predictions = {}
for block_id in y_block_data_test:
  print(block_id)
  accuracy = accuracy_score(y_block_data_test[block_id], y_block_data_pred[block_id])
  print("Accuracy:", accuracy)
  mse = mean_squared_error(y_block_data_test[block_id], y_block_data_pred[block_id])
  print("MSE:", mse)

Accuracy: 0.8333333333333334
0
Accuracy: 1.0
MSE: 0.0
1
Accuracy: 1.0
MSE: 0.0
2
Accuracy: 1.0
MSE: 0.0
3
Accuracy: 1.0
MSE: 0.0
4
Accuracy: 1.0
MSE: 0.0
5
Accuracy: 1.0
MSE: 0.0
6
Accuracy: 1.0
MSE: 0.0
7
Accuracy: 1.0
MSE: 0.0
8
Accuracy: 1.0
MSE: 0.0
9
Accuracy: 1.0
MSE: 0.0
10
Accuracy: 1.0
MSE: 0.0
11
Accuracy: 1.0
MSE: 0.0
12
Accuracy: 1.0
MSE: 0.0
13
Accuracy: 1.0
MSE: 0.0
14
Accuracy: 1.0
MSE: 0.0
15
Accuracy: 1.0
MSE: 0.0
16
Accuracy: 1.0
MSE: 0.0
17
Accuracy: 1.0
MSE: 0.0
18
Accuracy: 1.0
MSE: 0.0
19
Accuracy: 1.0
MSE: 0.0
20
Accuracy: 1.0
MSE: 0.0
21
Accuracy: 1.0
MSE: 0.0
22
Accuracy: 1.0
MSE: 0.0
23
Accuracy: 1.0
MSE: 0.0
24
Accuracy: 1.0
MSE: 0.0
25
Accuracy: 1.0
MSE: 0.0
26
Accuracy: 1.0
MSE: 0.0
27
Accuracy: 1.0
MSE: 0.0
28
Accuracy: 1.0
MSE: 0.0


###Testing on Block Level

In [33]:
# Create block_mapping_train
block_mapping_train = {}
for index, row in X_train.iterrows():  # Assuming X_train is your feature DataFrame
    window_id = row['Window_ID']
    block_id = row['Block_ID']
    block_mapping_train[window_id] = block_id

# Create block_mapping_test
block_mapping_test = {}
for index, row in X_test.iterrows():  # Assuming X_test is your feature DataFrame
    window_id = row['Window_ID']
    block_id = row['Block_ID']
    block_mapping_test[window_id] = block_id

# Group the testing dataset by 'Block' and aggregate the 'Target' variable
testing_ids = X_test['Window_ID'].unique()
test_data = data[data['Window_ID'].isin(testing_ids)]
actual_block_targets_test = test_data.groupby('Window_ID')['fatigue_rating'].mean().to_dict()

# Remove identifiers from data
X_test = X_test.drop(columns=['Window_ID'])
X_test = X_test.drop(columns=['Block_ID'])
X_train = X_train.drop(columns=['Window_ID'])
X_train = X_train.drop(columns=['Block_ID'])

Mean

In [34]:
from sklearn.tree import DecisionTreeRegressor
import numpy as np

# Sample dataset
# Assuming X_train, y_train, X_test, y_test are already prepared
# X_train and X_test should contain window-level features
# y_train and y_test should contain window-level target variable
# block_mapping_train and block_mapping_test should contain mapping between windows and blocks

# Train Decision Tree model
model = DecisionTreeRegressor()
model.fit(X_train, y_train)

# Predict on test set
window_predictions = model.predict(X_test)

# Aggregate window predictions to block level
block_predictions_mean = {}
for i, (window_pred, block_id) in enumerate(zip(window_predictions, block_mapping_test)):
    if block_id not in block_predictions_mean:
        block_predictions_mean[block_id] = []
    block_predictions_mean[block_id].append(window_pred)

# Average window predictions within each block
for block_id, window_preds in block_predictions_mean.items():
    block_predictions_mean[block_id] = np.mean(window_preds)

# Now block_predictions contains predictions for each block
print("Predictions at the block level:", block_predictions_mean)

Predictions at the block level: {'4_3_2': 86.0, '4_2_0': 82.0, '20_6_2': 14.0, '7_6_0': 100.0, '7_1_3': 48.0, '8_2_3': 59.0, '20_1_2': 14.0, '19_5_1': 60.0, '19_1_2': 40.0, '20_2_0': 11.0, '8_3_3': 67.0, '19_5_3': 60.0, '7_1_0': 48.0, '20_0_7': 13.0, '7_2_3': 87.0, '8_1_3': 41.0, '4_4_1': 84.0, '19_3_0': 50.0, '14_5_0': 64.0, '19_6_0': 48.0, '7_0_1': 43.0, '19_1_1': 40.0, '8_3_2': 67.0, '5_0_1': 55.0, '4_6_0': 84.0, '4_3_3': 86.0, '20_0_5': 13.0, '20_2_1': 11.0, '20_4_1': 7.0, '20_5_0': 9.0, '7_1_2': 48.0, '13_6_2': 100.0, '19_6_2': 48.0, '19_3_1': 50.0, '8_6_3': 76.0, '7_1_4': 48.0, '8_6_2': 76.0, '5_6_0': 70.0, '5_3_2': 59.0, '19_2_3': 47.0, '13_0_4': 86.0, '7_1_1': 48.0}


In [35]:
from sklearn.metrics import mean_squared_error, accuracy_score

# Convert block-level predictions and actuals to arrays
block_preds_array_mean = np.array(list(block_predictions_mean.values()))
actual_block_targets_array_mean = np.array(list(actual_block_targets_test.values()))

# Calculate Mean Squared Error (MSE) for regression tasks
mse = mean_squared_error(actual_block_targets_array_mean, block_preds_array_mean)
print("Mean Squared Error (MSE):", mse)

# Calculate Accuracy for classification tasks
accuracy = accuracy_score(actual_block_targets_array_mean, block_preds_array_mean)
print("Accuracy:", accuracy)


ValueError: Found input variables with inconsistent numbers of samples: [37, 42]

Mode

In [36]:
from sklearn.tree import DecisionTreeRegressor
import numpy as np
from scipy import stats

# Sample dataset
# Assuming X_train, y_train, X_test, y_test are already prepared
# X_train and X_test should contain window-level features
# y_train and y_test should contain window-level target variable
# block_mapping_train and block_mapping_test should contain mapping between windows and blocks

# Train Decision Tree model
model = DecisionTreeRegressor()
model.fit(X_train, y_train)

# Predict on test set
window_predictions = model.predict(X_test)

# Aggregate window predictions to block level
block_predictions_mode = {}
for i, (window_pred, block_id) in enumerate(zip(window_predictions, block_mapping_test)):
    if block_id not in block_predictions_mode:
        block_predictions_mode[block_id] = []
    block_predictions_mode[block_id].append(window_pred)

# Average window predictions within each block
for block_id, window_preds in block_predictions_mode.items():
    block_predictions_mode[block_id] = stats.mode(window_preds)[0]

# Now block_predictions contains predictions for each block
print("Predictions at the block level:", block_predictions_mode)

Predictions at the block level: {'4_3_2': 86.0, '4_2_0': 82.0, '20_6_2': 14.0, '7_6_0': 100.0, '7_1_3': 48.0, '8_2_3': 59.0, '20_1_2': 14.0, '19_5_1': 60.0, '19_1_2': 40.0, '20_2_0': 11.0, '8_3_3': 67.0, '19_5_3': 60.0, '7_1_0': 48.0, '20_0_7': 13.0, '7_2_3': 87.0, '8_1_3': 41.0, '4_4_1': 84.0, '19_3_0': 50.0, '14_5_0': 64.0, '19_6_0': 48.0, '7_0_1': 43.0, '19_1_1': 50.0, '8_3_2': 67.0, '5_0_1': 55.0, '4_6_0': 84.0, '4_3_3': 86.0, '20_0_5': 13.0, '20_2_1': 11.0, '20_4_1': 7.0, '20_5_0': 9.0, '7_1_2': 48.0, '13_6_2': 100.0, '19_6_2': 48.0, '19_3_1': 50.0, '8_6_3': 76.0, '7_1_4': 48.0, '8_6_2': 76.0, '5_6_0': 70.0, '5_3_2': 59.0, '19_2_3': 47.0, '13_0_4': 86.0, '7_1_1': 48.0}


In [37]:
from sklearn.metrics import mean_squared_error, accuracy_score

# Convert block-level predictions and actuals to arrays
block_preds_array_mode = np.array(list(block_predictions_mode.values()))
actual_block_targets_array_mode = np.array(list(actual_block_targets_test.values()))

# Calculate Mean Squared Error (MSE) for regression tasks
mse = mean_squared_error(actual_block_targets_array_mode, block_preds_array_mode)
print("Mean Squared Error (MSE):", mse)

# Calculate Accuracy for classification tasks
accuracy = accuracy_score(actual_block_targets_array_mode, block_preds_array_mode)
print("Accuracy:", accuracy)


ValueError: Found input variables with inconsistent numbers of samples: [37, 42]

Median

In [ ]:
from sklearn.tree import DecisionTreeRegressor
import numpy as np
from scipy import stats

# Sample dataset
# Assuming X_train, y_train, X_test, y_test are already prepared
# X_train and X_test should contain window-level features
# y_train and y_test should contain window-level target variable
# block_mapping_train and block_mapping_test should contain mapping between windows and blocks

# Train Decision Tree model
model = DecisionTreeRegressor()
model.fit(X_train, y_train)

# Predict on test set
window_predictions = model.predict(X_test)

# Aggregate window predictions to block level
block_predictions_median = {}
for i, (window_pred, block_id) in enumerate(zip(window_predictions, block_mapping_test)):
    if block_id not in block_predictions_median:
        block_predictions_median[block_id] = []
    block_predictions_median[block_id].append(window_pred)

# Average window predictions within each block
for block_id, window_preds in block_predictions_median.items():
    block_predictions_median[block_id] = np.median(window_preds)

# Now block_predictions contains predictions for each block
print("Predictions at the block level:", block_predictions_median)

In [ ]:
from sklearn.metrics import mean_squared_error, accuracy_score

# Convert block-level predictions and actuals to arrays
block_preds_array_median = np.array(list(block_predictions_median.values()))
actual_block_targets_array_median = np.array(list(actual_block_targets_test.values()))

# Calculate Mean Squared Error (MSE) for regression tasks
mse = mean_squared_error(actual_block_targets_array_median, block_preds_array_median)
print("Mean Squared Error (MSE):", mse)

# Calculate Accuracy for classification tasks
accuracy = accuracy_score(actual_block_targets_array_median, block_preds_array_median)
print("Accuracy:", accuracy)


Min

In [ ]:
from sklearn.tree import DecisionTreeRegressor
import numpy as np
from scipy import stats

# Sample dataset
# Assuming X_train, y_train, X_test, y_test are already prepared
# X_train and X_test should contain window-level features
# y_train and y_test should contain window-level target variable
# block_mapping_train and block_mapping_test should contain mapping between windows and blocks

# Train Decision Tree model
model = DecisionTreeRegressor()
model.fit(X_train, y_train)

# Predict on test set
window_predictions = model.predict(X_test)

# Aggregate window predictions to block level
block_predictions_min = {}
for i, (window_pred, block_id) in enumerate(zip(window_predictions, block_mapping_test)):
    if block_id not in block_predictions_min:
        block_predictions_min[block_id] = []
    block_predictions_min[block_id].append(window_pred)

# Average window predictions within each block
for block_id, window_preds in block_predictions_min.items():
    block_predictions_min[block_id] = np.min(window_preds)

# Now block_predictions contains predictions for each block
print("Predictions at the block level:", block_predictions_min)

In [ ]:
from sklearn.metrics import mean_squared_error, accuracy_score

# Convert block-level predictions and actuals to arrays
block_preds_array_min = np.array(list(block_predictions_min.values()))
actual_block_targets_array_min = np.array(list(actual_block_targets_test.values()))

# Calculate Mean Squared Error (MSE) for regression tasks
mse = mean_squared_error(actual_block_targets_array_min, block_preds_array_min)
print("Mean Squared Error (MSE):", mse)

# Calculate Accuracy for classification tasks
accuracy = accuracy_score(actual_block_targets_array_min, block_preds_array_min)
print("Accuracy:", accuracy)


Max

In [ ]:
from sklearn.tree import DecisionTreeRegressor
import numpy as np
from scipy import stats

# Sample dataset
# Assuming X_train, y_train, X_test, y_test are already prepared
# X_train and X_test should contain window-level features
# y_train and y_test should contain window-level target variable
# block_mapping_train and block_mapping_test should contain mapping between windows and blocks

# Train Decision Tree model
model = DecisionTreeRegressor()
model.fit(X_train, y_train)

# Predict on test set
window_predictions = model.predict(X_test)

# Aggregate window predictions to block level
block_predictions_max = {}
for i, (window_pred, block_id) in enumerate(zip(window_predictions, block_mapping_test)):
    if block_id not in block_predictions_max:
        block_predictions_max[block_id] = []
    block_predictions_max[block_id].append(window_pred)

# Average window predictions within each block
for block_id, window_preds in block_predictions_max.items():
    block_predictions_max[block_id] = np.max(window_preds)

# Now block_predictions contains predictions for each block
print("Predictions at the block level:", block_predictions_max)

In [ ]:
from sklearn.metrics import mean_squared_error, accuracy_score

# Convert block-level predictions and actuals to arrays
block_preds_array_max = np.array(list(block_predictions_max.values()))
actual_block_targets_array_max = np.array(list(actual_block_targets_test.values()))

# Calculate Mean Squared Error (MSE) for regression tasks
mse = mean_squared_error(actual_block_targets_array_max, block_preds_array_max)
print("Mean Squared Error (MSE):", mse)

# Calculate Accuracy for classification tasks
accuracy = accuracy_score(actual_block_targets_array_max, block_preds_array_max)
print("Accuracy:", accuracy)


###Information Gain Scores

In [38]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor

# Create and fit a Random Forest Regressor
rf = RandomForestRegressor(n_estimators=100, random_state=42)
rf.fit(X, y)

# Get feature importances
feature_importances = rf.feature_importances_

# Create a DataFrame to display feature names and their importances
feature_importance_df = pd.DataFrame({'Feature': X.columns, 'Importance': feature_importances})

# Sort features by importance
feature_importance = feature_importance_df.sort_values(by='Importance', ascending=False)
# Print or visualize feature importances
for i in range(len(feature_importance)):
  print(feature_importance["Feature"][i], feature_importance_df["Importance"][i])

Block 0.01038316742602564
window_num_60 2.791316471907416e-07
Age 0.034361886383514006
HRV_MeanNN 1.8622428026674775e-05
HRV_SDNN 0.0001257829308946049
HRV_RMSSD 4.4083902698406965e-06
HRV_SDSD 1.635746660836372e-05
HRV_CVNN 8.278187106707033e-06
HRV_CVSD 0.00015819330240271034
HRV_MedianNN 2.052338447012283e-05
HRV_MadNN 0.00013107168521522537
HRV_MCVNN 0.0007142143054975694
HRV_IQRNN 0.0002870911090123979
HRV_SDRMSSD 2.9379388586188673e-06
HRV_Prc20NN 0.00010197918604586804
HRV_Prc80NN 0.00017138714433456001
HRV_pNN50 5.91444147812679e-05
HRV_pNN20 9.285962008533936e-06
HRV_MinNN 8.843663015539685e-06
HRV_MaxNN 6.802945538659798e-06
HRV_HTI 2.187105614117416e-05
HRV_TINN 4.326958200225594e-05
HRV_LF 1.5204165735698657e-05
HRV_HF 1.6681948865526992e-05
HRV_VHF 8.089040068084643e-06
HRV_TP 7.987027950609509e-06
HRV_LFHF 0.0002593540805802109
HRV_LFn 5.164381390376746e-05
HRV_HFn 5.232527873327086e-05
HRV_LnHF 3.701580741341461e-06
HRV_SD1 5.5380987053701375e-05
HRV_SD2 3.92519886815135

In [39]:
feature_importance

,Feature,Importance
153,Block_ID,3.479327e-01
152,Window_ID,3.478372e-01
78,Change_from_Baseline,7.357210e-02
2,Age,3.436189e-02
145,Delta_HRV_MSEn,1.905522e-02
...,...,...
37,HRV_PIP,5.547662e-07
1,window_num_60,2.791316e-07
71,HRV_CMSEn,0.000000e+00
64,HRV_MFDFA_alpha1_Fluctuation,0.000000e+00
